In [25]:
from utils import get_answer, followup_prompt

In [37]:
function_name="lib_echo"
IS_C_CODE= True
lib_files = ['/home/sagemaker-user/afl-training/harness/library.c','/home/sagemaker-user/afl-training/harness/library.h']
HARNESS = 'harness.c'

In [27]:
lib_for_prompt = ''

code_md = '```c' if IS_C_CODE else '```c++'
for l in lib_files:
    with open(l) as file:
        content = f"\n{l}\n{code_md}\n{file.read()}\n```\n"
    lib_for_prompt = lib_for_prompt + content
lib_for_prompt=f'''
## Library:
{lib_for_prompt}
## End of library
'''

In [29]:

role = f"You are a developer. you need to write a libFuzzer target for the {function_name} function of your c library."

fill_c = '''
<FILL>

int LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {
'''
fill_cpp = '''
<FILL>

extern "C" int LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {
'''
template= '''
#include <stdint.h>
#include <stdlib.h>
'''+'''
// Step 2 include custom and standard headers
'''+ fill_c if IS_C_CODE else fill_c_cp+f'''
// Step 3 call to the {function_name} function
<FILL>;
''' +'''
  return 0;
}
'''
graph='}'
prompt = f"""
{role}

Work step by step:
1. Identify and analyze {function_name} in the library source code: which headers and parameters does it need to be properly executed?
2. Fill the template with necessary headers
3. fill the template LLVMFuzzerTestOneInput function with a call to {function_name}, such that {function_name} can be properly fuzz
Answer should be code only, no explanation.

## Library:
library.c
```c
{libc}
```
library.h
```c
{libh}
```
## End of library

## template:
```c
{template}
```
## End of template:
 """
prompt = '[INST]' + prompt + '[/INST]'

In [30]:
# payload = {
#     "inputs": "import socket\n\ndef ping_exponential_backoff(host: str):",
#     "parameters": {
#         "max_new_tokens": 256,
#         "top_p": 0.9,
#         "temperature": 0.2
#     }
# }
# response = predictor.predict(payload)
# print(response)

In [31]:
# print(response[0]['generated_text'])

In [32]:
r = get_answer(prompt)

========= PAYLOAD =========
payload = {
    "inputs": "[INST]\nYou are a developer. you need to write a libFuzzer target for the lib_echo function of your c library.\n\nWork step by step:\n1. Identify and analyze lib_echo in the library source code: which headers and parameters does it need to be properly executed?\n2. Fill the template with necessary headers\n3. fill the template LLVMFuzzerTestOneInput function with a call to lib_echo, such that lib_echo can be properly fuzz\nAnswer should be code only, no explanation.\n\n## Library:\nlibrary.c\n```c\n#include <stdlib.h>\n#include <stdio.h>\n#include <string.h>\n#include <assert.h>\n\n#include \"library.h\"\n\nvoid lib_echo(char *data, ssize_t len){\n\tif(strlen(data) == 0) {\n\t\treturn;\n\t}\n\tchar *buf = calloc(1, len);\n\tstrncpy(buf, data, len);\n\tprintf(\"%s\",buf);\n\tfree(buf);\n\n\t// A crash so we can tell the harness is working for lib_echo\n\tif(data[0] == 'p') {\n\t\tif(data[1] == 'o') {\n\t\t\tif(data[2] =='p') {\n\t\t

In [33]:
print(r[0]['generated_text'])

  ```
#include <stdint.h>
#include <stdlib.h>
#include "library.h"

int LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {
    char *data = (char *)Data;
    lib_echo(data, Size);
    return 0;
}
```


In [54]:

def extract_and_save_harness(llm_res):
    lines = llm_res[0]['generated_text'].split('\n')
    start_idx = next(i for i, line in enumerate(lines) if '```' in line) + 1
    end_idx = next(i for i, line in enumerate(lines) if i > start_idx and '```' in line)
    c_code = '\n'.join(lines[start_idx:end_idx])
    with open(HARNESS, 'w') as file:
        file.write(c_code)

In [71]:
import subprocess

compile_command = ['afl-gcc-fast', '-fsanitize=fuzzer', HARNESS, 'library.c', '-o', 'harness']

def compile(compile_command):
    result = subprocess.run(compile_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode == 0:
        print("Compilation successful.")
        return True, ""
    else:
        print("Compilation failed.")
        print("Errors:", result.stderr)
        # with open('error.log', 'w') as file:
        #     file.write(result.stderr)
        return False, result.stderr

def run_seed():
    result = subprocess.run([f'./harness', 'seed/seed'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode == 0:
        return True, ""
    else:
        print("Errors:", result.stderr)
        return False, result.stderr

In [72]:
# make sure that harness is compilable
tentatives = 10
prev_prompt = prompt
prev_res =  r[0]['generated_text']

# compile_command = ['afl-gcc-fast', '-fsanitize=fuzzer', HARNESS, 'library.c', '-o', 'file']


while tentatives:
    print(f"{tentatives=}")
    tentatives = tentatives -1
    compile_res = compile(compile_command)
    if compile_res[0]:
        run_res = run_seed()
        if run_res[0]:
            break
        else:
            print(f"{run_res[1]=}")
            prev_prompt = followup_prompt(prev_prompt, prev_res, "The generated code when run with input AAAA is crashing with this error\n" + run_res[1] + '\ncan you regenerate the previous code to fix the issue?')
            prev_res = get_answer(prev_prompt)
            extract_and_save_harness(prev_res)
    else:
        print(f"{compile_res[1]=}")
        prev_prompt = followup_prompt(prev_prompt, prev_res, "can you regenerate the previous code to fix the following compilation error:\n" + compile_res[1])
        prev_res = get_answer(prev_prompt)
        extract_and_save_harness(prev_res)
        # break

tentatives=10
Compilation successful.


In [ ]:
# checking that does not crash on seed
tentatives = 10

while tentatives:
    print(f"{tentatives=}")
    tentatives = tentatives -1
    exec_res = compile(compile_command)
    if compile_res[0]:
        break
    else:
        print(f"{compile_res[1]=}")
        prev_prompt = followup_prompt(prev_prompt, prev_res, "can you regenerate the previous code to fix the following error:\n" + compile_res[1])
        prev_res = get_answer(prev_prompt)
        extract_and_save_harness(prev_res)
        # break

In [44]:
# with open('error.log', 'r') as file:
#     content = file.read()
# fix_compilation_prompt = followup_prompt(prompt,  r[0]['generated_text'], "can you regenerate the previous code to fix the following error:\n" + content)

In [56]:
# r = get_answer(fix_compilation_prompt)

'<s>[INST]\nYou are a developer. you need to write a libFuzzer target for the lib_echo function of your c library.\n\nWork step by step:\n1. Identify and analyze lib_echo in the library source code: which headers and parameters does it need to be properly executed?\n2. Fill the template with necessary headers\n3. fill the template LLVMFuzzerTestOneInput function with a call to lib_echo, such that lib_echo can be properly fuzz\nAnswer should be code only, no explanation.\n\n## Library:\nlibrary.c\n```c\n\n#include <stdlib.h>\n#include <stdio.h>\n#include <string.h>\n#include <assert.h>\n\n#include "library.h"\n\nvoid lib_echo(char *data, ssize_t len){\n\tif(strlen(data) == 0) {\n\t\treturn;\n\t}\n\tchar *buf = calloc(1, len);\n\tstrncpy(buf, data, len);\n\tprintf("%s",buf);\n\tfree(buf);\n\n\t// A crash so we can tell the harness is working for lib_echo\n\tif(data[0] == \'p\') {\n\t\tif(data[1] == \'o\') {\n\t\t\tif(data[2] ==\'p\') {\n\t\t\t\tif(data[3] == \'!\') {\n\t\t\t\t\tassert(0)

In [ ]:
# make sure that the harness is fuzzable